# VacationPy
---


In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [2]:
# Loading the CSV file created in WeatherPy into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# View sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,port mathurin,-19.6833,63.4167,79.14,80,14,23.87,MU,1710617215
1,1,novodvinsk,64.4165,40.8122,32.79,99,100,8.66,RU,1710617215
2,2,waitangi,-43.9535,-176.5597,56.82,80,69,14.00,NZ,1710616996
3,3,fort bragg,35.1390,-79.0060,74.64,51,75,4.61,US,1710617020
4,4,pailon,-17.6500,-62.7500,95.90,41,92,15.50,BO,1710617217


---

### Creating Map with a point for every city in `city_data_df` DataFrame and humidity as size of point.

In [3]:
%%capture --no-display

# Configuring map plot
city_map = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    size = "Humidity",
    tiles = "OSM",
    color = "City",
    alpha = 0.5,
    frame_height = 500,
    frame_width = 700)


city_map

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Finding ideal weather condition

In [4]:
#setting ideal weather conditions for vacation
ideal_temp_range = (75, 95)
ideal_humidity = (0, 90)
# Narrowing down cities that fit criteria and drop any results with null values
ideal_temp_df = city_data_df.loc[(city_data_df["Max Temp"] >= ideal_temp_range[0]) &
                                  (city_data_df["Max Temp"] <= ideal_temp_range[1]) &
                                 (city_data_df["Humidity"] >= ideal_humidity[0]) &
                                 (city_data_df["Humidity"] <= ideal_humidity[1])]

# Droping any rows with null values
ideal_df = ideal_temp_df.dropna()

# View sample data
ideal_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,port mathurin,-19.6833,63.4167,79.14,80,14,23.87,MU,1710617215
6,6,almeirim,-1.5233,-52.5817,81.84,80,84,8.32,BR,1710617217
8,8,sao joao da barra,-21.6403,-41.0511,89.69,81,7,11.99,BR,1710617099
13,13,bouar,5.9340,15.5960,78.40,44,39,1.07,CF,1710617220
24,24,funtua,11.5233,7.3081,86.50,14,0,9.24,NG,1710617226


### Creatin new DataFrame, `hotel_df`.

In [5]:
# Copying previous df to create df 'hotel_df' to store the city, country, coordinates, and humidity
hotel_df = ideal_df.copy(deep=True)

# Adding empty column, "Hotel Name," to df to store the hotel found using Geoapify API
hotel_df.drop(["City_ID", "Max Temp", "Cloudiness", "Wind Speed", "Date"], axis = 1, inplace = True)
hotel_df["Hotel Name"] = ""
# View sample data
hotel_df.head(20)

,City,Lat,Lng,Humidity,Country,Hotel Name
0,port mathurin,-19.6833,63.4167,80,MU,
6,almeirim,-1.5233,-52.5817,80,BR,
8,sao joao da barra,-21.6403,-41.0511,81,BR,
13,bouar,5.9340,15.5960,44,CF,
24,funtua,11.5233,7.3081,14,NG,
28,utrik,11.2278,169.8474,71,MH,
32,san patricio,28.0170,-97.5169,51,US,
36,copala,16.6500,-98.9833,50,MX,
37,phang nga,8.4509,98.5298,77,TH,
40,lusambo,-4.9667,23.4500,81,CD,


### Finding first hotel within 10,000 metres of coordinates using Geoapify API for each city

In [6]:
#try 1 city first
lon_1 = -93.8363
lat_1 = 33.0932
base_url = "https://api.geoapify.com/v2/places"
categories = "accommodation.hotel"
radius = 50000
filter_circle = f"circle:{lon_1},{lat_1},{radius}"
bias = f"proximity:{lon_1},{lat_1}"
limit = len(hotel_df)
params = {"categories": categories,
          "filter": filter_circle,
          "bias": bias,
          "apiKey": geoapify_key}

response = requests.get(base_url,params=params)
response

<Response [200]>

In [7]:
response.url

'https://api.geoapify.com/v2/places?categories=accommodation.hotel&filter=circle%3A-93.8363%2C33.0932%2C50000&bias=proximity%3A-93.8363%2C33.0932&apiKey=88d3a2803165465eb637587fc0cb1010'

In [8]:
# Setting parameters to search for a hotel
radius = 10000
categories = "accommodation.hotel"
params = {"categories": categories,
          "filter": "",
          "bias": "",
          "apiKey": geoapify_key
}

# Printing message to follow up the hotel search
print("Starting hotel search")

# Iterating through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    #getting latitude, longitude from the DataFrame
    lat = hotel_df.loc[index]["Lat"]
    long = hotel_df.loc[index]["Lng"]

    # Adding filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{long},{lat},{radius}"
    params["bias"] = f"proximity:{long},{lat}"
    
    # Setting base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Making API request using the params dictionaty
    name_address = requests.get(base_url, params=params)
    
    # Converting the API response to JSON format
    name_address = name_address.json()
    
    # Grabbing the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# View sample data
hotel_df.head(20)

Starting hotel search
port mathurin - nearest hotel: Escale Vacances
almeirim - nearest hotel: Kapejar Hotel
sao joao da barra - nearest hotel: Pousada Mediterrâneo
bouar - nearest hotel: Mission catholique
funtua - nearest hotel: No hotel found
utrik - nearest hotel: No hotel found
san patricio - nearest hotel: No hotel found
copala - nearest hotel: No hotel found
phang nga - nearest hotel: Hotel
lusambo - nearest hotel: No hotel found
west island - nearest hotel: Cocos Village Bungalows
pimentel - nearest hotel: Hospedaje Calle del Sol
mount isa - nearest hotel: Ibis Styles
adamstown - nearest hotel: No hotel found
dogondoutchi - nearest hotel: Maison Alheri
fale old settlement - nearest hotel: No hotel found
puerto colombia - nearest hotel: Hotel Puerto Colombia
avarua - nearest hotel: Paradise Inn
mahibadhoo - nearest hotel: No hotel found
banda aceh - nearest hotel: Hotel 61
general villegas - nearest hotel: Rucalén
fada - nearest hotel: No hotel found
rongelap - nearest hotel: No

,City,Lat,Lng,Humidity,Country,Hotel Name
0,port mathurin,-19.6833,63.4167,80,MU,Escale Vacances
6,almeirim,-1.5233,-52.5817,80,BR,Kapejar Hotel
8,sao joao da barra,-21.6403,-41.0511,81,BR,Pousada Mediterrâneo
13,bouar,5.9340,15.5960,44,CF,Mission catholique
24,funtua,11.5233,7.3081,14,NG,No hotel found
28,utrik,11.2278,169.8474,71,MH,No hotel found
32,san patricio,28.0170,-97.5169,51,US,No hotel found
36,copala,16.6500,-98.9833,50,MX,No hotel found
37,phang nga,8.4509,98.5298,77,TH,Hotel
40,lusambo,-4.9667,23.4500,81,CD,No hotel found


### Adding hotel name and the country as additional information in the hover message for each city in the map.

In [9]:
%%capture --no-display

# Configuring the map plot
city_map_hotels = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    size = "Humidity",
    tiles = "OSM",
    color = "City",
    alpha = 0.5,
    hover_cols=["Hotel Name", "Country"],
    frame_height = 500,
    frame_width = 700)


city_map_hotels

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)